# 2.1.1 Functions

This is a small introduction to functions from the point of view of functional programming _and_ software engineering. It ends with a discussion about the basic way in which functions are represented in an object-oriented programming language like Scala. 

### References

_[Optional]_ __[Why Functional Programming Matters](https://www.cs.kent.ac.uk/people/staff/dat/miranda/whyfp90.pdf)__ John Hughes. This is a classic paper that motivates the need for functional programming by appealing to software enginerring principles such as modularity. It's written using the Miranda programming language, and it's more an academic paper that undergraduate material. Nevertheless, its reading is highly recommeded. Give it a try if you find some time!

__[Scala book (online)](https://docs.scala-lang.org/overviews/scala-book/introduction.html)__.

- [Pure functions](https://docs.scala-lang.org/overviews/scala-book/pure-functions.html)

## What are functions?

Functions are computational devices that transform input _values_ into output _values_, and do nothing _else_.

In [1]:
// `add one` function

def add(input: Int): Int = 
  input + 1

defined function add

If we run this function, the only thing that happens is the computation of a new value:

In [2]:
add(3)

res1: Int = 4

Functions that do something else, besides returning values, are called _impure_ functions. Functional programming deals only with _pure_, or mathematical, functions.

In [3]:
// An impure function
def impureAdd(input: Int): Int = {
    println("adding one to " + input)
    input + 1
}

defined function impureAdd

If we run this function, we will see an _effect_ in the console (besides the pure computation of `input + 1`): 

In [4]:
impureAdd(3)

adding one to 3


res3: Int = 4

There are many kinds of effects: writing to the console, reading from the keyboard, reading from a socket, calling a web service, executing a query over the database, etc. Clearly, we need effects if we want our programs to do something useful, so pure functions alone are not enough. We will talk about this later on.



## Functions as modularity devices

Why are functions so important in programming? Because they help us to _modularize_ our code. For instance, let's consider the following programs, which access the following data structure of key-value pairs (we will talk about this structure in detail later on):

In [5]:
val config: Map[String, String] = 
    Map("URL" -> "http://hablapps.com",
        "PORT" -> "8080")

config: Map[String, String] = Map(
  "URL" -> "http://hablapps.com",
  "PORT" -> "8080"
)

Our first program access the configuration data for the value of the "URL" key. If it's not found, then the default value "default.url" is returned (similarly, we will discuss the `match` keyword further in the course).

In [6]:
// Program 1
val url: String = config.get("URL") match {
  case Some(u) => u
  case None => "default.url"
}

url: String = "http://hablapps.com"

Our second program accesses the configuration data for the value of the "PORT" key. If it's not found, then the default value "8080" is returned.

In [7]:
// Program 2
val port: String = config.get("PORT") match {
  case Some(p) => p
  case None => "8080"
}

port: String = "8080"

These two programs do _almost_ the same. The only differences lie in the particular keys and default values the programs refer to, but, otherwise, they do the same thing. However, this _common factor_ is not reflected in the code. Indeed, we may get one program from the other by copy-pasting, a clear signal of [code-smell](https://en.wikipedia.org/wiki/Code_smell).

These programs are _monolythic_, in the sense that they are not made by composing large enough modules. In this case, the common logic of the program and the values it operates on are intermingled in the same code. 

How can we abstract away the differences and package the common logic in a single module? With functions:

In [8]:
/*
val port: String = config.get("PORT") match {
  case Some(p) => p
  case None => "8080"
}
*/
def getKeyFrom(
    config: Map[String, String], 
    key: String, 
    default: String): String =
  config.get(key) match {
    case Some(p) => p
    case None => default
  }

defined function getKeyFrom

This is an abstract module which we can combine with other modules to get back the very same functionality:

In [9]:
// Program 1
val url: String = getKeyFrom(config, "URL", "default.url")

url: String = "http://hablapps.com"

In this case, we combine the module `getKeyFrom` with the modules (data values and variables, in particular) `config`, `"URL"` and `"default.url"`. The composition method is just simple function application.

Which are the advantages of using functions? As in the general case, having a more modular solution enables _reuse_, particularly of those modules which are abstract or parameterised. For instance, we can benefit from this level of reuse by re-implementing the `url` program in the following way:

In [10]:
// Program 2
val port: String = getKeyFrom(config, "PORT", "8080")

port: String = "8080"

## Functions as methods

In an object-oriented language, functions are implemented through _methods_, i.e. using the `def` keyword. Note that these methods are invariably part of an `object`, `class` or `trait` declaration. Typically, pure functions are declared as part of objects. For instance, we may declare a set of arithmetic functions as follows: 

In an object-oriented language, functions are primarily implemented through _methods_, i.e. using the `def` keyword. These methods are invariably part of an `object`, `class` or `trait` declaration. For instance, we may declare a set of arithmetic functions as follows: 

In [11]:
import scala.math.{pow, Pi}

object Areas{
    
    def circle(radius: Double): Double = 
        Pi*pow(radius, 2)
    
    def rectangle(width: Double, height: Double): Double = 
        width * height
}

import scala.math.{pow, Pi}


defined object Areas

In notebooks and the Scala REPL, `def` declarations appear to be independent from any object or class, but they are not:

In [12]:
def foo(i: Int): Int = i
// show errors: "missing argument list for method foo in class Helper"
// foo

defined function foo

## Functions as values

Functions can also be represented as _values_, i.e. as objects. This allows us to implement functions that receive other functions as arguments, or return functions as results. This special functions are called _higher-order functions_ (HOF), and they feature as a great modularity device. We will mainly discuss this feature of HOFs in PF-3; now, we just want to focus on how are functions actually represented as values in a OO language like Scala. 

This representation builds essentially upon methods, in particular, _reified_ methods. For instance, let's consider the following functions:

In [15]:
def addOneM(number: Int): Int = 
    number + 1

def substractOneM(number: Int): Int = 
    number - 1 

defined function addOneM
defined function substractOneM

We want to implement a HOF that receives an integer-to-integer function, such as `addOneM`and `substractOneM`, and calls this function over a given number. We may want to write something like this:

In [15]:
// def call(def int2int(n: Int): Int, number: Int): Int =
//   int2int(number)

where the first argument `int2int` attempts to represent any function that receives an integer and returns another integer. 

This code is not legal in Scala, but we can create a new class whose only method is the function that we want to pass around:

In [16]:
abstract class FunctionInt2Int{
    def apply(number: Int): Int
}

defined class FunctionInt2Int

Now, we can implement the `call` HOF as follows: 

In [17]:
def call(int2int: FunctionInt2Int, number: Int): Int = 
    int2int.apply(number)

defined function call

In order to use this HOF with the `addOneM` and `substractOneM` functions, we must create reified versions for them: 

In [18]:
val addOneV: FunctionInt2Int = new FunctionInt2Int{
    def apply(number: Int): Int = 
        number + 1
}

val substractOneV: FunctionInt2Int = new FunctionInt2Int{
    def apply(number: Int): Int = 
        number - 1 
}

addOneV: FunctionInt2Int = ammonite.$sess.cmd17$Helper$$anon$1@1f91fcec
substractOneV: FunctionInt2Int = ammonite.$sess.cmd17$Helper$$anon$2@3f2a1dd5

We call the `addOneV` and `substractOneV` function-values, i.e. functions represented as values. Now, we can use the `call` HOF as follows:

In [19]:
assert(call(addOneV, 5) == 6)

In [20]:
call(substractOneV, 5)

res19: Int = 4

## Standard functions in Scala

The Scala programming language offers many facilities to work with functions as values. First, the standard library provides the following _generic_ types [`Function1`](https://www.scala-lang.org/api/current/scala/Function1.html), [`Function2`](https://www.scala-lang.org/api/current/scala/Function2.html), etc., roughly implemented as follows:

In [21]:
object Std{
    trait Function1[A, B]{
        def apply(a: A): B
    }

    trait Function2[A, B, C]{
        def apply(a: A, b: B): C
    }

    // up to Function22
}


defined object Std

Using these standard classes, we can create the `addOneV` function-value in a similar way than before: 

In [22]:
val addOneV: Function1[Int, Int] = new Function1[Int, Int]{
    def apply(a: Int): Int = 
        a + 1
}

addOneV: Int => Int = <function1>

But we can do it more easily, since Scala also provides special syntax to declare function types and instantiate functions (so-called _lambda expressions_):

In [23]:
val addOneV: Int => Int = 
    (a: Int) => a + 1

val substractOneV: Int => Int = 
    (a: Int) => a - 1

addOneV: Int => Int = ammonite.$sess.cmd22$Helper$$Lambda$2150/0x0000000100b53840@1a46a15f
substractOneV: Int => Int = ammonite.$sess.cmd22$Helper$$Lambda$2151/0x0000000100b53040@221a0dd3

And we can also profit from type inference:

In [24]:
val addOneV: Int => Int = 
    a => a + 1

val substractOneV: Int => Int = 
    a => a - 1

addOneV: Int => Int = ammonite.$sess.cmd23$Helper$$Lambda$2158/0x0000000100b69840@7a4942be
substractOneV: Int => Int = ammonite.$sess.cmd23$Helper$$Lambda$2159/0x0000000100b6a040@73caa9be

Using these syntactic facilities, the `call` HOF has a more appealing signature: 

In [25]:
def call(int2int: Int => Int, number: Int): Int = 
    // int2int.apply(number)
    int2int(number)

defined function call

which we can use as follows:

In [26]:
call(addOneV, 1)
call(substractOneV, 1)

res25_0: Int = 2
res25_1: Int = 0

And we can even pass function-methods that are converted on the fly to function-values!

In [27]:
call(addOneM, 1)
call(substractOneM, 1)

res26_0: Int = 2
res26_1: Int = 0

This is the so-called _eta-expansion_.

Last, we can get extra level of conciseness using so-called _underscore_ syntax:

In [28]:
val addOne: Int => Int = 
    _ + 1

addOne: Int => Int = ammonite.$sess.cmd27$Helper$$Lambda$2177/0x0000000100b74840@232a62e

In [29]:
val times: (Int, Int) => Int = 
    _ * _

times: (Int, Int) => Int = ammonite.$sess.cmd28$Helper$$Lambda$2182/0x0000000100b77840@7d1f92ec

In [30]:
call(_ + 1, 5)
call(_ - 1, 3)

res29_0: Int = 6
res29_1: Int = 2

## Currying

We can use a similar syntax for functions of two arguments. So, the function-value equivalent of this function-method: 

In [31]:
def sumM(x: Int, y: Int): Int = 
    x+y

defined function sumM

could be written in this way: 

In [32]:
val sumV: Function2[Int, Int, Int] = new Function2[Int, Int, Int]{
    def apply(a: Int, b: Int): Int = 
         a + b
}

sumV: (Int, Int) => Int = <function2>

but also in the following one:

In [ ]:
val sumV: (Int, Int) => Int = 
    (a: Int, b: Int) => a + b

or, exploiting type inference:

In [ ]:
val sumV: (Int, Int) => Int = 
    (a, b) => a + b 

However, `Function2`, `Function3`, ... classes are not extrictly necessary, and sometimes we can get along with `Function1`. But, how can we create a function of two arguments with `Function1` alone? The trick is the following:

In [ ]:
def sumC: Int => (Int => Int) = 
    (a: Int) => (b: Int) => a + b : Int

Note that brackets in `Int => (Int => Int)` are used for clarity, but are not needed. Basically, we created a function of one argument that returns another function of one argument. So, the expression: 

In [ ]:
sumC(1): (Int => Int)

returns a function that can be applied again:

In [ ]:
sumC(1)(2)

We can apply this strategy to functions of any number of arguments. This is called _currying_ and _currified functions_. The analog in function-methods is [multiple-parameter lists](https://docs.scala-lang.org/tour/multiple-parameter-lists.html):

In [ ]:
def sum(x: Int)(y: Int): Int = 
    x+y

## Functions compose

We can create new functions by composing other functions whose signatures match. This is great from a modularity perspective. For instance, the following function is implemented in a non-modular way:

In [ ]:
def isEvenLength: String => Boolean = 
    (s: String) => s.length % 2 == 0

This function is somehow the combination of two more basic functions `length` and `isEven`:

In [ ]:
def length: String => Int = 
    s => s.length

In [ ]:
def isEven: Int => Boolean = 
    i => i % 2 == 0

but this is not reflected in the current implementation. How can we redefine the function `isEvenLength` using the functions `length` and `isEven`? We can use a HOF which helps us to compose functions:

In [ ]:
def compose[A, B, C](f2: B => C, f1: A => B): A => C = 
    (a: A) => f2(f1(a))

Then, we can redefine `isEvenLength` in a modular way from the `length` and `isEven` building blocks:

In [ ]:
val isEvenLength: String => Boolean = 
    compose(isEven, length)

The HOF `compose` is actually defined by `Function1`: 

In [ ]:
val isEvenLength: String => Boolean = 
    isEven.compose(length)

or using infix notation:

In [ ]:
val isEvenLength: String => Boolean = 
    isEven compose length

Note that a similar function to `compose`, called `andThen`, is also available in the standard library: 

In [ ]:
val isEvenLength: String => Boolean = 
    length andThen isEven

The last implementation of the `compose` HOF was _monomorphic_, in the sense that it only works with specific types. We can obtain a more flexible implementation using generics, also known as _parametric polymorphism_:

In [ ]:
def compose[A, B, C](f2: B => C, f1: A => B): A => C = 
    (a: A) => f2(f1(a))

We can also give a currified version of this function as follows:

In [ ]:
def compose[A, B, C]: (B => C) => (A => B) => A => C = 
    f2 => f1 => (a: A) => f2(f1(a))

Last, there is a function which behaves as the identity element with respect to the operation `compose`, i.e. no matter which other function we choose to compose with the [`identity`](https://www.scala-lang.org/api/current/scala/Predef$.html) function, the result will be that function:
1. `identity[B] compose f == f` for all `f: A => B`
2. `f compose identity[A] == f` for all `f: A => B`

In [ ]:
def identity[A](a: A): A = a

or using lambda expressions:

In [ ]:
def identity[A]: A => A = 
    (a: A) => a

## Contravariance and variance in `Function` traits

Trait `Function1` is actually defined as follows:

In [ ]:
object StdFunctions{
    trait Function1[-A, +B]{
        def apply(a: A): B
    }
}

Given the following hierarchy:

In [ ]:
abstract class Animal
class Perro extends Animal
class Doberman extends Perro
class Gato extends Animal
class Siames extends Gato

In [ ]:
def fGP: Gato => Perro = ??? 
def fGD: Gato => Doberman = ??? 
def fGA: Gato => Animal = ???
def fAP: Animal => Perro = ???
def fSP: Siames => Perro = ??? 
def fAD: Animal => Doberman = ???

which functions can be passed to this method?

In [ ]:
def f2(f: Gato => Perro): Unit = ()

In [ ]:
def r1 = f2(fGP)
def r2 = f2(fGD)
//def r3 = f2(fGA)
def r4 = f2(fAP)
//def r5 = f2(fSP)
def r6 = f2(fAD)